In this section, we will create some aggregated tables to enable analysis in SQL, I used duckdb lib to enable SQL queries in python notebook.

The approach is to build purpose-specific aggregate tables for the analysis we need rather than single big 'everything joined' table because we are dealing with multiple bridging tables - we can easily double count everything

In [1]:
import duckdb, os, glob

con = duckdb.connect(database='/content/sql_analysis.duckdb', read_only=False)

In [14]:

CLEAN = "/content/data/clean"

# --- Sales ---
con.execute(f"""
CREATE OR REPLACE TABLE silver_sales AS
SELECT
  CAST(OrderID AS BIGINT) AS OrderID,
  CAST(OrderDate AS DATE) AS OrderDate,
  CAST(CustomerID AS BIGINT) AS CustomerID,
  CAST(StoreID AS BIGINT) AS StoreID,
  CAST(BundledProductID AS BIGINT)   AS BundledProductID,
  CAST(SalesAmount AS DECIMAL(18,2)) AS SalesAmount
FROM read_csv_auto('{CLEAN}/sales.csv');
""")


In [15]:
con.execute(f"""
SELECT *
FROM silver_sales
""").df().head()

,OrderID,OrderDate,CustomerID,StoreID,BundledProductID,SalesAmount
0,1001,2024-01-15,5001,101,3001,150.0
1,1002,2024-01-16,5002,102,3002,200.0
2,1003,2024-01-17,5003,101,3003,300.0
3,1004,2024-02-15,5001,103,3001,100.0
4,1005,2024-02-16,5002,101,3002,250.0


In [16]:
# --- PRODUCTS ---
con.execute(f"""
CREATE OR REPLACE TABLE silver_products AS
SELECT
  CAST(ProductID AS BIGINT) AS ProductID,
  CAST(CategoryID AS BIGINT) AS CategoryID,
  CAST(Price AS DECIMAL(18,2)) AS Price,
  CAST(ProductName AS VARCHAR) AS ProductName
FROM read_csv_auto('{CLEAN}/products.csv');
""")

# --- CATEGORIES ---
con.execute(f"""
CREATE OR REPLACE TABLE silver_categories AS
SELECT
  CAST(CategoryID AS BIGINT)  AS CategoryID,
  CAST(CategoryName AS VARCHAR)   AS CategoryName
FROM read_csv_auto('{CLEAN}/categories.csv');
""")

# --- CUSTOMERS ---
con.execute(f"""
CREATE OR REPLACE TABLE silver_customers AS
SELECT
  CAST(CustomerID AS BIGINT)  AS CustomerID,
  CAST(Name AS VARCHAR)   AS Name,
  CAST(Gender AS VARCHAR)         AS Gender,
  CAST(Age AS INTEGER)        AS Age,
  CAST(City AS VARCHAR)           AS City
FROM read_csv_auto('{CLEAN}/customers.csv');
""")

# --- STORES ---
con.execute(f"""
CREATE OR REPLACE TABLE silver_stores AS
SELECT
  CAST(StoreID AS BIGINT) AS StoreID,
  CAST(StoreName AS VARCHAR) AS StoreName,
  CAST(Location AS VARCHAR) AS Location,
  CAST(Manager AS VARCHAR) AS Manager
  FROM read_csv_auto('{CLEAN}/stores.csv');
""")

# --- BUNDLED PRODUCTS ---
con.execute(f"""
CREATE OR REPLACE TABLE silver_bundledproducts AS
SELECT
  CAST(BundledProductID AS BIGINT) AS BundledProductID,
  CAST(BundledProductName AS VARCHAR) AS BundledProductName
FROM read_csv_auto('{CLEAN}/bundledproducts.csv');
""")

# --- BRIDGE: BundledProduct Products ---
con.execute(f"""
CREATE OR REPLACE TABLE silver_bundledproducts_products AS
SELECT
  CAST(BP_Prod_BridgeID AS BIGINT) AS BP_Prod_BridgeID,
  CAST(BundledProductID AS BIGINT) AS BundledProductID,
  CAST(ProductID AS BIGINT) AS ProductID
FROM read_csv_auto('{CLEAN}/bundledproducts_products.csv');
""")

# --- BRIDGE: BundledProducts Promotions ---
con.execute(f"""
CREATE OR REPLACE TABLE silver_bundlepromotion AS
SELECT
  CAST(BP_Promo_BridgeID AS BIGINT) AS BP_Promo_BridgeID,
  CAST(BundledProductID AS BIGINT)  AS BundledProductID,
  CAST(PromotionID AS BIGINT) AS PromotionID
FROM read_csv_auto('{CLEAN}/bundledproductspromotions.csv');
""")

# --- PROMOTIONS ---
con.execute(f"""
CREATE OR REPLACE TABLE silver_promotions AS
SELECT
  CAST(PromotionID AS BIGINT)        AS PromotionID,
  CAST(PromotionName AS VARCHAR)         AS PromotionName,
  CAST(StartDate AS DATE) AS StartDate,
  CAST(EndDate AS DATE) AS EndDate,
  CAST(DiscountPercentage AS DECIMAL(9,4)) AS DiscountPercentage
FROM read_csv_auto('{CLEAN}/promotions.csv');
""")

print(con.execute("SHOW TABLES;").df())
print(con.execute("SELECT COUNT(*) AS n_rows, MIN(OrderDate) AS min_dt, MAX(OrderDate) AS max_dt FROM silver_sales;").df())
print(con.execute("SELECT COUNT(*) AS n_rows, MIN(StartDate) AS min_start, MAX(EndDate) AS max_end FROM silver_promotions;").df())

                              name
0           silver_bundledproducts
1  silver_bundledproducts_products
2           silver_bundlepromotion
3                silver_categories
4                 silver_customers
5                  silver_products
6                silver_promotions
7                     silver_sales
8                    silver_stores
   n_rows     min_dt     max_dt
0     129 2024-01-15 2026-02-23
   n_rows  min_start    max_end
0      12 2024-01-01 2024-12-31


**CREATE some views to enable analysis**

View 1: v_sales_product_weighted

This is to eenable the analysis on sales for bundles and product.

However, each order only at the bundle level,  there is no product level sales stored. To obtain the sales amount for products, we will need to make an assumption on sales per product, the approach here I am using is to calculate the weight of products based on their listed price in [products] table


In [45]:
con.execute("""
CREATE OR REPLACE VIEW v_sales_product_weighted AS
with bundle_products AS (
  SELECT
    bp.BundledProductID,
    bp.ProductID,
    p.Price AS ProductPrice
  FROM silver_bundledproducts_products bp
  JOIN silver_products p
    ON p.ProductID = bp.ProductID
),

bundle_price_sums AS (
  SELECT
    BundledProductID,
    SUM(ProductPrice) AS BundleTotalPrice
  FROM bundle_products
  GROUP BY BundledProductID
),

sales_price_weighted AS (
  SELECT
    s.OrderID,
    s.OrderDate,
    s.BundledProductID,
    bp.ProductID,
    bp.ProductPrice,
    bps.BundleTotalPrice,
    s.SalesAmount,
    CASE
      WHEN bps.BundleTotalPrice IS NULL OR bps.BundleTotalPrice = 0 THEN
        1.0 / COUNT(*) OVER (PARTITION BY s.OrderID, s.BundledProductID)
      -- handle missing values if bundle total price is null or 0, so evenly split the products in the bundle
      ELSE bp.ProductPrice / bps.BundleTotalPrice
    END AS ProductShare,
    CASE
      WHEN bps.BundleTotalPrice IS NULL OR bps.BundleTotalPrice = 0 THEN
        s.SalesAmount * (1.0 / COUNT(*) OVER (PARTITION BY s.OrderID, s.BundledProductID))
      ELSE s.SalesAmount * (bp.ProductPrice / bps.BundleTotalPrice)
    END AS ProductSalesAmount
  FROM silver_sales s
  JOIN bundle_products bp
    ON s.BundledProductID = bp.BundledProductID
  LEFT JOIN bundle_price_sums bps
    ON s.BundledProductID = bps.BundledProductID
)

SELECT
  s.OrderID,
  s.OrderDate,
  s.BundledProductID,
  s.SalesAmount,
  s.ProductID,
  s.ProductPrice,
  s.ProductShare,
  s.ProductSalesAmount,
  b.BundledProductName,
  p.ProductName,
  c.CategoryID,
  c.CategoryName,
  st.StoreID,
  st.StoreName,
  cu.CustomerID,
  cu.Name
FROM sales_price_weighted s
LEFT JOIN silver_bundledproducts b ON s.BundledProductID = b.BundledProductID
LEFT JOIN silver_products p        ON s.ProductID = p.ProductID
LEFT JOIN silver_categories c      ON p.CategoryID = c.CategoryID
LEFT JOIN silver_stores st         ON st.StoreID = (SELECT StoreID FROM silver_sales WHERE OrderID = s.OrderID LIMIT 1)
LEFT JOIN silver_customers cu      ON cu.CustomerID = (SELECT CustomerID FROM silver_sales WHERE OrderID = s.OrderID LIMIT 1);
""")


,Count


In [48]:
con.execute("""
SELECT *
FROM v_sales_product_weighted
LIMIT 10;
""").df().head(5)

,OrderID,OrderDate,BundledProductID,SalesAmount,ProductID,ProductPrice,ProductShare,ProductSalesAmount,BundledProductName,ProductName,CategoryID,CategoryName,StoreID,StoreName,CustomerID,Name
0,1017,2024-08-23,3002,500.0,2006,400.0,0.470588,235.294118,Bundle B,Product F,100,Electronics,102,Store B,5001,John Doe
1,1017,2024-08-23,3002,500.0,2005,350.0,0.411765,205.882353,Bundle B,Product E,104,Books,102,Store B,5001,John Doe
2,1017,2024-08-23,3002,500.0,2004,100.0,0.117647,58.823529,Bundle B,Product D,103,Sports,102,Store B,5001,John Doe
3,1083,2025-03-31,3006,300.0,2014,800.0,0.533333,160.000000,Bundle F,Product N,103,Sports,101,Store A,5011,Patricia Taylor
4,1083,2025-03-31,3006,300.0,2013,700.0,0.466667,140.000000,Bundle F,Product M,102,Home & Kitchen,101,Store A,5011,Patricia Taylor


Analysis 1: Sales per products

In [49]:
con.execute("""
SELECT ProductID, ProductName, SUM(ProductSalesAmount) AS TotalSales
FROM v_sales_product_weighted
GROUP BY 1,2
ORDER BY TotalSales DESC;
""").df()

,ProductID,ProductName,TotalSales
0,2016,Product P,6950.000000
1,2007,Product G,5343.750000
2,2005,Product E,4885.026738
3,2010,Product J,4830.000000
4,2012,Product L,3954.545455
5,2002,Product B,3416.783217
6,2011,Product K,3295.454545
7,2009,Product I,3220.000000
8,2008,Product H,3206.250000
9,2001,Product A,3200.769231


One thing to note here is - When Product does not have listed price, we are assuming it to be non-revenue items and are assigned no share of sales revenue. However again, the missing values need to align with business rules



Analysis 2: Monthly trend for sales amount

Also checking if the total product sales match with total bundle sales

In [60]:
con.execute("""
SELECT
  DATE_TRUNC('month', OrderDate) AS Month,
  SUM(ProductSalesAmount) AS ProductSales,
  SUM(CASE WHEN rn = 1 THEN SalesAmount END) AS BundleSales
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY OrderID, BundledProductID ORDER BY ProductID) AS rn
  FROM v_sales_product_weighted
)
GROUP BY 1
ORDER BY Month;
""").df()


,Month,ProductSales,BundleSales
0,2024-01-01,650.0,650.0
1,2024-02-01,350.0,350.0
2,2024-03-01,750.0,750.0
3,2024-04-01,3050.0,3050.0
4,2024-05-01,2700.0,2700.0
5,2024-06-01,3100.0,3100.0
6,2024-07-01,2600.0,2600.0
7,2024-08-01,1950.0,1950.0
8,2024-09-01,4000.0,4000.0
9,2024-10-01,1750.0,1750.0


View 2: v_promotion_impact

This is to enable analysis on promotion impact

Even though in our dataset, one bundle is only linked to one promotion, however in reality there is possibility one bundle can be linked to multiple promotion and promotions were active during the order date, so in the SQL logic we also assume if promotion is active and there are multiple promotion, we will pick the highest promotion

In [74]:
con.execute("""
CREATE OR REPLACE VIEW v_promotion_impact AS
WITH promo_map AS (
  SELECT
    bp.BundledProductID,
    pr.PromotionID,
    pr.PromotionName,
    pr.DiscountPercentage AS DiscountPct,
    pr.StartDate,
    pr.EndDate
  FROM silver_bundlepromotion bp
  JOIN silver_promotions pr ON pr.PromotionID = bp.PromotionID
),
candidates AS (
  SELECT
    v.*,
    pm.PromotionID,
    pm.PromotionName,
    pm.DiscountPct,
    pm.StartDate,
    pm.EndDate,
    CASE WHEN v.OrderDate BETWEEN pm.StartDate AND pm.EndDate THEN 1 ELSE 0 END AS IsPromotionActive,
    -- Rank: active promos first, then highest discount
    ROW_NUMBER() OVER (
      PARTITION BY v.OrderID, v.BundledProductID, v.ProductID
      ORDER BY
        CASE WHEN v.OrderDate BETWEEN pm.StartDate AND pm.EndDate THEN 1 ELSE 0 END DESC,
        pm.DiscountPct DESC NULLS LAST
    ) AS rn
  FROM v_sales_product_weighted v
  LEFT JOIN promo_map pm
    ON pm.BundledProductID = v.BundledProductID
),
chosen AS (
  SELECT * FROM candidates WHERE rn = 1 OR rn IS NULL
)
SELECT
  OrderID,
  OrderDate,
  BundledProductID,
  BundledProductName,
  ProductID,
  ProductName,
  CategoryID,
  CategoryName,
  StoreID,
  StoreName,
  CustomerID,
  Name,
  ProductShare,
  ProductSalesAmount AS GrossProductSalesAmount,
  CASE WHEN IsPromotionActive = 1 AND DiscountPct IS NOT NULL
       THEN DiscountPct ELSE 0 END AS AppliedDiscountPct,

  CASE WHEN IsPromotionActive = 1 AND DiscountPct IS NOT NULL
       THEN ProductSalesAmount * (DiscountPct / 100.0)
       ELSE 0 END AS DiscountAmount,

  ProductSalesAmount
    - CASE WHEN IsPromotionActive = 1 AND DiscountPct IS NOT NULL
           THEN ProductSalesAmount * (DiscountPct / 100.0)
           ELSE 0 END AS NetProductSalesAmount,

  PromotionID,
  PromotionName,
  DiscountPct,
  IsPromotionActive,
  CASE
    WHEN PromotionID IS NULL                THEN 'No Promotion'
    WHEN IsPromotionActive = 1              THEN 'In Promotion'
    ELSE 'Outside Promo Period'
  END                                                     AS PromotionStatus
FROM chosen;
""")


In [75]:
con.execute("""
SELECT *
FROM v_promotion_impact
LIMIT 10;
""").df()

,OrderID,OrderDate,BundledProductID,BundledProductName,ProductID,ProductName,CategoryID,CategoryName,StoreID,StoreName,...,ProductShare,GrossProductSalesAmount,AppliedDiscountPct,DiscountAmount,NetProductSalesAmount,PromotionID,PromotionName,DiscountPct,IsPromotionActive,PromotionStatus
0,1075,2025-02-03,3005,Bundle E,2012,Product L,101,Clothing,106,Store F,...,0.545455,81.818182,0.0,0.0,81.818182,4005,Promo E,30.0,0,Outside Promo Period
1,1096,2025-06-30,3004,Bundle D,2010,Product J,104,Books,104,Store D,...,0.600000,210.000000,0.0,0.0,210.000000,4004,Promo D,25.0,0,Outside Promo Period
2,1102,2025-08-11,3005,Bundle E,2012,Product L,101,Clothing,107,Store G,...,0.545455,81.818182,0.0,0.0,81.818182,4005,Promo E,30.0,0,Outside Promo Period
3,1118,2025-12-01,3003,Bundle C,2007,Product G,101,Clothing,104,Store D,...,0.625000,156.250000,0.0,0.0,156.250000,4003,Promo C,20.0,0,Outside Promo Period
4,1085,2025-04-14,3001,Bundle A,2002,Product B,101,Clothing,101,Store A,...,0.307692,169.230769,0.0,0.0,169.230769,4001,Promo A,10.0,0,Outside Promo Period
5,1086,2025-04-21,3005,Bundle E,2012,Product L,101,Clothing,103,Store C,...,0.545455,163.636364,0.0,0.0,163.636364,4005,Promo E,30.0,0,Outside Promo Period
6,1088,2025-05-05,3005,Bundle E,2011,Product K,100,Electronics,101,Store A,...,0.454545,318.181818,0.0,0.0,318.181818,4005,Promo E,30.0,0,Outside Promo Period
7,1090,2025-05-19,3002,Bundle B,2006,Product F,100,Electronics,104,Store D,...,0.470588,329.411765,0.0,0.0,329.411765,4002,Promo B,15.0,0,Outside Promo Period
8,1094,2025-06-16,3007,Bundle G,2015,Product O,104,Books,105,Store E,...,NaN,NaN,0.0,0.0,NaN,4007,Promo G,15.0,0,Outside Promo Period
9,1105,2025-09-01,3009,Bundle I,2005,Product E,104,Books,106,Store F,...,0.636364,381.818182,0.0,0.0,381.818182,4009,Promo I,25.0,0,Outside Promo Period


Analysis 1: Monthly gross vs discount vs net

In [79]:
con.execute("""
SELECT
  DATE_TRUNC('month', OrderDate) AS Month,
  SUM(GrossProductSalesAmount)   AS GrossRevenue,
  SUM(DiscountAmount)            AS DiscountValue,
  SUM(NetProductSalesAmount)     AS NetRevenue
FROM v_promotion_impact
GROUP BY 1
ORDER BY 1;
""").df()

,Month,GrossRevenue,DiscountValue,NetRevenue
0,2024-01-01,650.0,15.0,635.0
1,2024-02-01,350.0,37.5,312.5
2,2024-03-01,750.0,80.0,670.0
3,2024-04-01,3050.0,150.0,2900.0
4,2024-05-01,2700.0,0.0,2700.0
5,2024-06-01,3100.0,0.0,3100.0
6,2024-07-01,2600.0,187.5,2412.5
7,2024-08-01,1950.0,0.0,1950.0
8,2024-09-01,4000.0,0.0,4000.0
9,2024-10-01,1750.0,0.0,1750.0


Total Promotion sales vs. Outside window sales

In [85]:
con.execute("""
SELECT
  PromotionID,
  PromotionName,
  AVG(CASE WHEN IsPromotionActive=1 THEN DiscountPct END) AS AvgAppliedDiscountPct,
  SUM(CASE WHEN IsPromotionActive=1 THEN NetProductSalesAmount END) AS PromoSales,
  SUM(CASE WHEN IsPromotionActive=0 AND PromotionID IS NOT NULL THEN GrossProductSalesAmount END) AS OutsideWindowSales
FROM v_promotion_impact
GROUP BY 1,2
ORDER BY PromoSales DESC;
""").df()

,PromotionID,PromotionName,AvgAppliedDiscountPct,PromoSales,OutsideWindowSales
0,4007,Promo G,15.0,1062.5,5700.0
1,4004,Promo D,25.0,450.0,7450.0
2,4003,Promo C,20.0,320.0,8150.0
3,4002,Promo B,15.0,212.5,4350.0
4,4001,Promo A,10.0,135.0,5400.0
5,4008,Promo H,NaN,NaN,3200.0
6,4009,Promo I,NaN,NaN,4700.0
7,4006,Promo F,NaN,NaN,6000.0
8,4005,Promo E,NaN,NaN,7250.0


Promotion Impact — Sales, Orders, and Average per Order

In [95]:
con.execute("""
SELECT
  BundledProductID,
  BundledProductName,

  SUM(CASE WHEN IsPromotionActive = 1 THEN NetProductSalesAmount ELSE 0 END) AS PromoRevenue,
  SUM(CASE WHEN IsPromotionActive = 0 OR PromotionID IS NULL THEN NetProductSalesAmount ELSE 0 END) AS NoPromoRevenue,

  COUNT(DISTINCT CASE WHEN IsPromotionActive = 1 THEN OrderID END) AS PromoOrders,
  COUNT(DISTINCT CASE WHEN IsPromotionActive = 0 OR PromotionID IS NULL THEN OrderID END) AS NoPromoOrders,

  CASE WHEN COUNT(DISTINCT CASE WHEN IsPromotionActive = 1 THEN OrderID END) > 0
       THEN SUM(CASE WHEN IsPromotionActive = 1 THEN NetProductSalesAmount ELSE 0 END)
            / COUNT(DISTINCT CASE WHEN IsPromotionActive = 1 THEN OrderID END)
       ELSE 0 END AS AvgSalesPerPromoOrder,

  CASE WHEN COUNT(DISTINCT CASE WHEN IsPromotionActive = 0 OR PromotionID IS NULL THEN OrderID END) > 0
       THEN SUM(CASE WHEN IsPromotionActive = 0 OR PromotionID IS NULL THEN NetProductSalesAmount ELSE 0 END)
            / COUNT(DISTINCT CASE WHEN IsPromotionActive = 0 OR PromotionID IS NULL THEN OrderID END)
       ELSE 0 END AS AvgSalesPerNoPromoOrder

FROM v_promotion_impact
GROUP BY BundledProductID, BundledProductName
ORDER BY PromoRevenue DESC;
""").df()


,BundledProductID,BundledProductName,PromoRevenue,NoPromoRevenue,PromoOrders,NoPromoOrders,AvgSalesPerPromoOrder,AvgSalesPerNoPromoOrder
0,3007,Bundle G,1062.5,5700.0,2,13,531.25,438.461538
1,3004,Bundle D,450.0,7450.0,1,16,450.00,465.625000
2,3003,Bundle C,320.0,8150.0,1,22,320.00,370.454545
3,3002,Bundle B,212.5,4350.0,1,11,212.50,395.454545
4,3001,Bundle A,135.0,5400.0,1,16,135.00,337.500000
5,3009,Bundle I,0.0,4700.0,0,11,0.00,427.272727
6,3006,Bundle F,0.0,6000.0,0,12,0.00,500.000000
7,3005,Bundle E,0.0,7250.0,0,17,0.00,426.470588
8,3008,Bundle H,0.0,3200.0,0,5,0.00,640.000000
